#PORTFÓLIÓ

Ezen notebookban portófiliót készítűnk a modellezett predikciók alapján. Összesen 30 részvényen lett futtatva a GRU modell, ezek részvények tickerei a stock_to_choose_from tömbben találhatóak. Azért, hogy a választható részvények halmaza ne ugyanaz legyen minden esetben, ezen 30 közül lesz kiválasztva STOCKS_TO_CHOOSE_FROM számú részvény, amelyek közül minden nap STOCK_TO_INVEST számú részvényegység kerül be a portfólióba.

A következő kódrészletek az adatok előkészítéséről szólnak.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from os import listdir
from sklearn.preprocessing import MinMaxScaler
import random

In [ ]:
STOCKS_TO_CHOOSE = 20

In [ ]:
all_dfs = []
stock_path = '../input/price-volume-data-for-all-us-stocks-etfs/Stocks'
stock_to_choose_from = ['aapl', 'abb', 'atvi', 'axp', 'cl', 'dis', 'f', 'gm', 'gme', 'goog',
                        'has', 'jnj', 'ko', 'mcd', 'mmm', 'msft', 'nflx', 'nke', 'nvda', 'orcl',
                        'pep', 'pfe', 'pm', 'sbux', 't', 'tmus', 'tsla', 'ual', 'v', 'wmt']
for stock in stock_to_choose_from:
    all_dfs.append({"name" : stock,
    "data" : pd.read_csv(os.path.join('../input/price-volume-data-for-all-us-stocks-etfs/Stocks',stock + '.us.txt'),delimiter=',',usecols=['Date','Open','High','Low','Close'])})
random.shuffle(all_dfs)
all_dfs = all_dfs[:STOCKS_TO_CHOOSE]

In [ ]:
TEST_INTERVAL = 200
TIMESTAMP = 50
NUM_OF_RUNS = 10

In [ ]:
test_datas = []
real_stock_price_list = []
predicted_stock_price_list = []
for df_data in all_dfs:
    df = df_data["data"]
    df = df[df['Date'].str.contains("2012") | df['Date'].str.contains("2013") | df['Date'].str.contains("2014") | df['Date'].str.contains("2015") | df['Date'].str.contains("2016") ]
    high_prices = df.loc[:,'High'].to_numpy()
    low_prices = df.loc[:,'Low'].to_numpy()
    mid_prices = (high_prices+low_prices)/2.0

    data_num = len(mid_prices)
    data_num_train = data_num - TEST_INTERVAL
    train_data = mid_prices[:data_num_train]
    test_data = mid_prices[data_num_train-TIMESTAMP:]
    test_datas.append(test_data)
    
    real_stock_price = test_data[TIMESTAMP:]
    real_stock_price_list.append({"name" : df_data["name"], "data" : real_stock_price})
    predicted_stock_prices = np.zeros((NUM_OF_RUNS,TEST_INTERVAL))
    predicted_stock_price_list.append({"name" : df_data["name"], "data" :predicted_stock_prices})
    

In [ ]:
real_stock_price_profit_per_day_list = []
predicted_stock_price_profit_per_day_list = []
profits = 0
for real_stock_price in real_stock_price_list:
    real_stock_price_profit_per_day = []
    real_stock_price = real_stock_price["data"]
    for i in range(1, TEST_INTERVAL):
        real_stock_price_profit_per_day.append((real_stock_price[i] - real_stock_price[0])/real_stock_price[0] * 100)

    predicted_stock_price_profits_per_day = [[0 for j in range(TEST_INTERVAL)] for i in range(NUM_OF_RUNS)]
    real_stock_price_profit_per_day_list.append(real_stock_price_profit_per_day)
    predicted_stock_price_profit_per_day_list.append(predicted_stock_price_profits_per_day)
    profits = profits + real_stock_price_profit_per_day[TEST_INTERVAL - 2]
 

In [ ]:
for predicted_stock_price in predicted_stock_price_list:
    predicted_stock_price["data"] = np.loadtxt(os.path.join('../input/stock-predictions-gru',predicted_stock_price["name"] + '_pred.csv'), delimiter=",")

In [ ]:
for all_df in all_dfs:
    print(all_df['name'], end =" ")

In [ ]:
all_stock_num = len(predicted_stock_price_list)
STOCKS_TO_INVEST = 12

In [ ]:
def calculatePortofolio(real_stock_price_list):
    columns = []
    for real_stock_price in real_stock_price_list:
        columns.append(real_stock_price["name"])
    portofolio_df = pd.DataFrame([], columns = columns)
    for i in range(1, TEST_INTERVAL):
        row = []
        for stock_idx in range(0, len(predicted_stock_price_list)):
            diffStock = real_stock_price_list[stock_idx]["data"]
            BUY_PRED_NUM = 0
            for j in range(NUM_OF_RUNS):
                predicted_stock_price = predicted_stock_price_list[stock_idx]["data"][j][i]
                if(predicted_stock_price>diffStock[i-1]):
                    BUY_PRED_NUM = BUY_PRED_NUM + 1
            row.append(BUY_PRED_NUM)
        df_row = pd. Series(row, index = portofolio_df.columns)
        #print(df_row)
        portofolio_df = portofolio_df.append(df_row, ignore_index=True)
    
    return portofolio_df

#PORTFÓLIÓ KÉSZÍTÉS

A portfólió készítést az egyes modellek összegzett predikciói alapján végezzük el. Minden napra kiválasztunk a STOCK_TO_CHOOSE_FROM számú részvény közül STOCKS_TO_INVEST-et, azokat amelyek a 10 modell közül, a legtöbb szerint nőni fognak. A top_stocks tömb tárolja, hogy mely részvényeknek a legnagyobb az esélye, hogy nőjjön. Ha nincs ilyen típusú részvény a jelenlegi részvényeink között akkor veszünk, ha van, akkor megtartjuk. Ha egy részvény, amelybe jelenleg van befektetésünk, kikerül a top_stocks tömbből, eladjuk.

Ha a paraméterben lévő weighted paraméter értéke False, akkor minden top_stocks-ban lévő részvényből ugyanannyit vásárolunk, ha ezen paraméter értéke True, akkor arányosan vásárolunk, annak függvényében, hogy hány modell prediktál növekedést (tehát, ha például a 10 modellből 8 prediktál növekedést 8/10 egységnyi részvényünk kell legyen összesen az adott részvényből). Többszöri futtatás, próbálás után, arra a következtetésre jutottam, hogy a súlyozás gyengíti az eredményességet, így ez nincs részletesen dokumentálva.

Az így léterhozott portfólió tipikusan akkor ér el jobb eredményeket a valódi növekedésnél, ha az összes részvénynek nem több mint 50%-ákat vesszük be a portfólió halmazba, és a valódi növekedés, nem jelentős (10% alatt van). Ezeket az eredményeket az alábbi ábrák is reprezentálják:

**STOCKS_TO_INVEST=3**

<a href="https://imgbb.com/"><img src="https://i.ibb.co/WBPgd7Z/image.png" alt="image" border="0"></a><br /><a target='_blank' href='https://imgbb.com/'>image uploader</a><br />

**STOCKS_TO_INVEST=5**

<a href="https://imgbb.com/"><img src="https://i.ibb.co/M8159Pb/image.png" alt="image" border="0"></a>

Végső soron azt a következtetést tudjuk levonnni, hogy mint az evaluations.ipynb-be bemutatott módszerek, mint a portólió generálásnál alkalamzott módszerekkel, egy konzervatív módszert dolgoztunk ki, mellyel (a tesztelt adatok alapján) kissebb az esély a veszteségre.


In [ ]:
def calculateProfitPortofolio(df, weighted):
    prev_buy_dates = np.zeros(all_stock_num)
    diffs = 0
    #for weighted case
    profits_per_stock = np.zeros(all_stock_num)
    num_of_bought_stocks = np.zeros(all_stock_num)
    for day in range(1, TEST_INTERVAL):
        predic_list = df.iloc[day-1]
        top_preds = np.zeros(STOCKS_TO_INVEST)
        top_stocks = np.empty(STOCKS_TO_INVEST,dtype="<U10")
        #for unweighted case
        buy_dates = np.zeros(all_stock_num)
        for stock_idx in range(all_stock_num):
            top_preds_idx = STOCKS_TO_INVEST - 1
            while(predic_list[stock_idx]>top_preds[top_preds_idx] and top_preds_idx>-1):
                top_preds_idx = top_preds_idx - 1
            if(top_preds_idx<STOCKS_TO_INVEST-1):
                for shift_idx in range(STOCKS_TO_INVEST-2, top_preds_idx, -1):
                    top_preds[shift_idx + 1] = top_preds[shift_idx]
                    top_stocks[shift_idx + 1] = top_stocks[shift_idx]
                top_preds[top_preds_idx+1] = predic_list[stock_idx]
                top_stocks[top_preds_idx+1] = df.columns[stock_idx]
        if(not weighted):    
            for stock_idx in range(all_stock_num):
                if(df.columns[stock_idx] in top_stocks):
                    if(prev_buy_dates[stock_idx] == 0):
                        buy_dates[stock_idx] = day
                    else:
                        buy_dates[stock_idx] = prev_buy_dates[stock_idx]
                else:
                    if(not prev_buy_dates[stock_idx] == 0):
                        prev_buy_date = int(prev_buy_dates[stock_idx])
                        current_value = real_stock_price_list[stock_idx]["data"][day]
                        buy_value = real_stock_price_list[stock_idx]["data"][prev_buy_date]
                        diff = ((current_value / buy_value) - 1) * 100
                        diffs = diffs + diff
                    buy_dates[stock_idx] = 0
            prev_buy_dates = buy_dates
        else:
            for stock_idx in range(all_stock_num):
                if(df.columns[stock_idx] in top_stocks): 
                    tops_idx = np.where(top_stocks == df.columns[stock_idx])
                    num_of_stocks_to_have = top_preds[tops_idx]
                    profits_per_stock[stock_idx] = profits_per_stock[stock_idx] + (num_of_bought_stocks[stock_idx] - num_of_stocks_to_have)*real_stock_price_list[stock_idx]["data"][day]
                    num_of_bought_stocks[stock_idx] = num_of_stocks_to_have
                else:
                    #print(profits_per_stock[stock_idx])
                    #print(num_of_bought_stocks[stock_idx]*real_stock_price_list[stock_idx]["data"][day])
                    profits_per_stock[stock_idx] = profits_per_stock[stock_idx] + num_of_bought_stocks[stock_idx]*real_stock_price_list[stock_idx]["data"][day]
                    num_of_bought_stocks[stock_idx] = 0
                    
        
        #print(day)
        #print(predic_list)
        #print(top_preds)
        #print(top_stocks)
        #if(weighted):
        #    print(num_of_bought_stocks)
        #    print(profits_per_stock)
    if(weighted):
        for stock_idx in range(all_stock_num):
            profits_per_stock[stock_idx] = profits_per_stock[stock_idx] + num_of_bought_stocks[stock_idx]*real_stock_price_list[stock_idx]["data"][TEST_INTERVAL-1]
            #print(profits_per_stock[stock_idx])
            #print(real_stock_price_list[stock_idx]["data"][0])
            diff = (profits_per_stock[stock_idx] / real_stock_price_list[stock_idx]["data"][0]) * 100 / 10
            diffs = diffs +  diff
    print(diffs/STOCKS_TO_INVEST)

In [ ]:
portofolio_df = calculatePortofolio(real_stock_price_list)

In [ ]:
calculateProfitPortofolio(portofolio_df, False)

In [ ]:
calculateProfitPortofolio(portofolio_df, True)

In [ ]:
profits/all_stock_num